# Exercise 6: Code Execution vs Tool Calling

Here, we will use the `LocalPythonExecutor` class from `smolagents` to implement an agent that writes code to solve problems.


In [ ]:
%pip install smolagents
%restart_python

In [ ]:
import pandas as pd
from langchain.agents import create_agent
from langchain.messages import AIMessage, HumanMessage
from langchain.tools import tool
from llm import model as llm
from smolagents import LocalPythonExecutor
from smolagents.local_python_executor import InterpreterError

df = pd.read_csv(
    "https://raw.githubusercontent.com/datasciencedojo/datasets/refs/heads/master/titanic.csv"
)


interpreter = LocalPythonExecutor(
    additional_authorized_imports=[
        "pandas",
        "matplotlib",
        "sklearn",
        "sklearn.linear_model",
        "matplotlib.pyplot",
        "datetime",
    ],
)
# here we could pass additional functions to the interpreter
interpreter.send_tools({})
interpreter.state["df"] = df
interpreter("import matplotlib; matplotlib.use('Agg'); import matplotlib.pyplot as plt")


# Exercise 6.1: Define the `exec_python` tool
# This tool should take a code string, execute it using the `interpreter`, and return the output.
# Make sure to handle `InterpreterError`.
# <solution>
# TODO: Implement this
pass
# </solution>


# Exercise 6.2: Create the agent
# Use `create_agent` with the model and tools.
# <solution>
# TODO: Implement this
pass
# </solution>

In [ ]:
# Let's test the agent
response = agent.invoke(
    {
        "messages": [
            HumanMessage(
                # "can you make some descriptive plots of the data and summarize it? avoid seaborn. save the plot"
                "can you make fit a logistc reg for who survived? the data is in df. do not stop until done."
            )
        ]
    }
)

# Print the message history
for msg in response["messages"]:
    if type(msg) is AIMessage and msg.tool_calls:
        print("=" * 20 + " Code " + "=" * 20)
        print(msg.tool_calls[0]["args"]["code"])
        print("=" * 50)
    print(msg.content)
    print("\n")

In [ ]:
# Exercise 6.3: Implement Memory Tools
# We want to give the agent a "long-term memory" to store results and errors.
# We will use LangGraph's InMemoryStore with semantic search capabilities.
#
# 1. Initialize `memory_store` with the embeddings.
# 2. Implement `create_memory(key: str, value: str, category: str)` to add to the store.
# 3. Implement `recall_memory(query: str)` to search the store using semantic search.

import uuid

from langgraph.store.memory import InMemoryStore
from llm import embedding_dimensions, embeddings

# <solution>
# TODO: Implement this
pass
# </solution>

In [ ]:
# Exercise 6.4: Create an Agent with Memory
# Create a new agent that has access to `exec_python`, `create_memory`, and `recall_memory`.
# Ask it to perform a calculation, store the result, and then in a follow-up ("later"), ask it to recall it.

# <solution>
# TODO: Implement this
pass
# </solution>

In [ ]:
# Test the memory agent
print("--- Testing Memory Agent ---")

# Step 1: Perform a task and remember the result
response1 = memory_agent.invoke(
    {
        "messages": [
            HumanMessage(
                "Calculate the mean age of passengers in the Titanic dataset (variable 'df'). "
                "Store the result in your memory so you don't have to calculate it again."
            )
        ]
    }
)
print(response1["messages"][-1].content)

# Step 2: Ask a question that requires recalling the memory
print("\n--- Testing Recall ---")
response2 = memory_agent.invoke(
    {
        "messages": [
            HumanMessage(
                "Do you remember the mean age of the passengers? "
                "Try recalling past analyses."
            )
        ]
    }
)
print(response2["messages"][-1].content)

In [ ]:
# Print the message history
for msg in response2["messages"]:
    if type(msg) is AIMessage and msg.tool_calls:
        print("=" * 20 + " Code " + "=" * 20)
        if "code" in msg.tool_calls[0]["args"]:
            print(msg.tool_calls[0]["args"]["code"])
        print("=" * 50)
    print(msg.content)
    print("\n")